In [5]:
import sympy.physics.mechanics as me
from symmeplot.pyqtgraph import Scene3D
import pyqtgraph as pg
import pyqtgraph.opengl as gl

class JupyterGLW(gl.GLViewWidget):
    def get_frame(self):
        # rather than eating up cpu cycles by perpetually updating "Updating plot",
        # we will only update it opportunistically on a redraw.
        self.request_draw()
        # update()
        return super().get_frame()
        
pg.mkQApp()
view = JupyterGLW()
N = me.ReferenceFrame("N")
A = me.ReferenceFrame("A")
A.orient_axis(N, N.z, 1)
N0 = me.Point("N_0")
v = 0.2 * N.x + 0.2 * N.y + 0.7 * N.z
A0 = N0.locatenew("A_0", v)
scene = Scene3D(N, N0, scale=0.5, view=view)
scene.add_vector(v, name="v")
scene.add_frame(A, A0, as_mesh=True)
scene.lambdify_system(())
scene.evaluate_system()
scene.plot()
display(view)

In [1]:
import sympy as sm
import sympy.physics.mechanics as me
import matplotlib.pyplot as plt
import matplotlib

me.init_vprinting()
%matplotlib inline

In [20]:
q1, u1 = me.dynamicsymbols('q1 u1') # q is the angle of the ellipse
q1d = me.dynamicsymbols('q1', 1)

x, ux = me.dynamicsymbols('x ux') # ccordinate, speed of contact point
xd = me.dynamicsymbols('x', 1)

m, g, a, b, iXX, iYY, iZZ, iXY, iXZ, iYZ = sm.symbols('m, g, a, b, iXX, iYY, iZZ, iXY, iXZ, iYZ')
reibung, t = sm.symbols('reibung t')
mx, my = me.dynamicsymbols('mx my') # geometric center of the ellipse

alfa, beta = sm.symbols('alfa beta')
y = sm.symbols('y') # y coordinate of CP. Is zero by definition
x1, y1, x2, y2 = sm.symbols('x1 y1 x2 y2') # auxuliary coordinates to for the ellipses

N = me.ReferenceFrame('N') # Inertial frame
A1 = me.ReferenceFrame('A1') # fixed to the ellipse

P0 = me.Point('P0') # fixed reference point
CP = me.Point('CP') # contact point of ball and plane
Dmc = me.Point('Dmc') # geometric center of the ball
m_Dmc = me.Point('m_Dmc') # mass center

A1.orient_axis(N, q1, N.z)
A1.set_ang_vel(N, u1*N.z)

CP.set_pos(P0, x*N.x) # contact point of ellipse with horizontal line
Dmc.set_pos(P0, mx*N.x + my*N.y)
P0.set_vel(N, 0)


In [1]:
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting()

L, k, m_node, m_elem = sm.symbols('L, k, m_{node}, m_{elem}')
F = me.dynamicsymbols('F')
d12, d13, d23 = me.dynamicsymbols('d_{12}, d_{13}, d_{23}')
x1, x2, x3, y1, y2, y3 = me.dynamicsymbols('x1:4, y1:4')
x1d, x2d, x3d, y1d, y2d, y3d = me.dynamicsymbols('dx1:4, dy1:4')
t = me.dynamicsymbols._t
# x2, y2, x2d, y2d = 0, 0, 0, 0

N = me.ReferenceFrame('N')
# O = me.Point('O')
# P1 = O.locatenew('p1', x1 * N.x + y1 * N.y)
# P1.set_vel(N, x1d * N.x + y1d * N.y)
P1 = me.Point('P1')
P1.set_vel(N, 0)
P2 = P1.locatenew('P2', x2 * N.x + y2 * N.y)
P2.set_vel(N, x2d * N.x + y2d * N.y)
P3 = P1.locatenew('P3', x3 * N.x + y3 * N.y)
P3.set_vel(N, x3d * N.x + y3d * N.y)

em12 = P1.locatenew('e_{12,mass}', P2.pos_from(P1)/2)
em13 = P1.locatenew('e_{13,mass}', P3.pos_from(P1)/2)
em23 = P2.locatenew('e_{23,mass}', P3.pos_from(P2)/2)

# Define masses and inertias
r_P1_P2_norm = P2.pos_from(P1).normalize()
F12 = (k * (P2.pos_from(P1).magnitude() - L) +
       d12 * (P2.vel(N).dot(r_P1_P2_norm) - P1.vel(N).dot(r_P1_P2_norm))) \
      * r_P1_P2_norm
r_P1_P3_norm = P3.pos_from(P1).normalize()
F13 = (k * (P3.pos_from(P1).magnitude() - L) +
       d13 * (P3.vel(N).dot(r_P1_P3_norm) - P1.vel(N).dot(r_P1_P3_norm))) \
      * r_P1_P3_norm
r_P2_P3_norm = P3.pos_from(P2).normalize()
F23 = (k * (P3.pos_from(P2).magnitude() - L) +
       d23 * (P3.vel(N).dot(r_P2_P3_norm) - P2.vel(N).dot(r_P2_P3_norm))) \
      * r_P2_P3_norm

points = [P2, P3, em12, em13, em23]
forces = [F*N.y - F12 + F23, F13 - F23, 0*N.x, 0*N.x, 0*N.x]
masses = [m_node, m_node, m_elem, m_elem, m_elem]

q = sm.Matrix([x2, x3, y2, y3])
u = sm.Matrix([x2d, x3d, y2d, y3d])
qdot_to_u = {qi.diff(): ui for qi, ui in zip(q, u)}
qddot_to_udot = {qi.diff(t, 2): ui.diff for qi, ui in zip(q, u)}
repl = qdot_to_u | qddot_to_udot

Fr_bar = []
Frs_bar = []

# loop over the three generalized speeds
for ur in u:

    # initialize the rth GAF and GIF
    Fr = 0
    Frs = 0

    # for the rth generalized speed, loop though each point to find it's
    # contribution to the generalized forces
    for Pi, Ri, mi in zip(points, forces, masses):
        vr = Pi.vel(N).subs(repl).diff(ur, N)  # rth partial velocity
        Fr += vr.dot(Ri)  # sum in Pi's contribution to GAF
        Rs = -mi*Pi.acc(N).subs(repl)  # rth inertia force
        Frs += vr.dot(Rs)  # sum in Pi's contribution to GIF

    Fr_bar.append(Fr)
    Frs_bar.append(Frs)
Fr = sm.Matrix(Fr_bar)
Frs = sm.Matrix(Frs_bar)

ud = u.diff()
Md = Frs.jacobian(ud)
ud_zerod = {udr: 0 for udr in ud}
gd = Frs.xreplace(ud_zerod) + Fr
gk = u
Mk = -sm.eye(len(q))
gm = sm.Matrix(sm.BlockMatrix([gk.T, gd.T])).T
Mm = sm.Matrix(sm.BlockMatrix([[Mk, sm.zeros(len(u))], [sm.zeros(len(u)), Md]]))
xm = sm.Matrix.hstack(u.T, u.diff().T).T

In [2]:
# display(Mm)
# display(xm)
# display(sm.simplify(gm))

In [3]:
particles = [me.Particle(p.name, p, mass=m) for p, m in zip(points, masses)]
forceslist = [(p, f) for p, f in zip(points, forces)]
kdes = list(u-q.diff())
KM = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kdes)
(fr, frstar) = KM.kanes_equations(particles, forceslist)
MM = KM.mass_matrix
forcing = KM.forcing
rhs = MM.inv() * forcing

In [5]:
sm.cse(rhs)

⎛                                                                             
⎜⎡                              ⎛       ___________⎞                          
⎜⎢                              ⎜      ╱   2     2 ⎟             ⎛    1 ⎞     
⎜⎢(x₀, -L), (x₁, x₃), (x₂, y₃), ⎝x₃, ╲╱  x₁  + x₂  ⎠, (x₄, dx₃), ⎜x₅, ──⎟, (x₆
⎜⎢                                                               ⎝    x₃⎠     
⎜⎣                                                                            
⎝                                                                             

                                                                              
                                                                              
                                                                              
, x₁⋅x₅), (x₇, dy₃), (x₈, x₂⋅x₅), (x₉, k⋅(x₀ + x₃) + (x₄⋅x₆ + x₇⋅x₈)⋅d_{13}), 
                                                                              
                                                   